In [1]:
import os, sys
# root = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# sys.path.append(root)

import random
import wandb
import pickle
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from data import rxn
import tqdm




/home/kguo2/anaconda3/envs/pygeo/lib/python3.9/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:
reaxys_mean = 64.05
reaxys_median = 68.00
reaxys_std = 22.46
# preparing data
folder = "./data/datav6_internal"
rxns = rxn(folder)
ids = rxns.all_idx2()
print(f'Total number of reactions: {len(ids)}')
for i, id in enumerate(ids):
    fp = rxns.get_fp_fast(id)
    aev = rxns.get_aev_fast(id)
    qm = rxns.get_qm_fast(id)
    steric = rxns.get_steric_desps(id)
    mordred = rxns.get_mordred_fast(id)
    yld = rxns.get_yield(id)
    print(yld)
    print(len(fp), len(aev), len(qm), len(steric), len(mordred))
    # print(aev)
    # print(qm)
    # print(steric)
    break

In [3]:
train_ids0 = pickle.load(open(os.path.join(folder, 'normal_ids.pkl'), 'rb'))
train_uncertain_ids = pickle.load(open(os.path.join(folder, 'train_uncertain_ids.pkl'),'rb'))
# train_ids0 = train_ids0.union(train_uncertain_ids)
test_ids = pickle.load(open(os.path.join(folder, 'test_clean_ids.pkl'), 'rb'))
test_ids_u = pickle.load(open(os.path.join(folder, 'test_uncertain_ids.pkl'), 'rb'))

In [4]:
def get_raw_feature(ids,reaction,standardize=True):
    X, Y = [], []
    molecule = True
    global input_dim
    for i,id in enumerate(ids):
        context = rxns.get_context_one_hot(id)
        fp = rxns.get_fp(id)
        # fp = rxns.get_fp_fast(id)
        # print(len(fp))
        context_fp0 = list(np.array(context + fp)) # 3807
        aimnet0 = rxns.get_qm_fast(id)  #51
        mordred0 = rxns.get_mordred(id)  #4944
        aev0 = rxns.get_aev_fast(id) # 5148
        # x = aimnet0 + context_fp0 + mordred0 + aev0 
        x = fp
        if reaction & molecule:
            reaction_x = [x[i + 2048] - x[i] - x[i + 1024] for i in range(1024)]        
            
            input_dim = len(x)//3*4
            combined_x = x + reaction_x
            X.append(combined_x)
            y = rxns.get_yield(id)
            Y.append(y)
        elif reaction:
            reaction_x = [x[i + 2048] - x[i] - x[i + 1024] for i in range(1024)]        
            input_dim = len(x)//3
            X.append(reaction_x)
            y = rxns.get_yield(id)
            Y.append(y)
            
        else:
            X.append(x)
            input_dim = len(x)
            y = rxns.get_yield(id)
            Y.append(y)
    X = np.array(X)
    Y = np.array(Y)

    return (X, Y)

In [ ]:
yhat = rg.predict(test_X) 
# make the max item in the item to be 1
binary_array = np.zeros_like(yhat)  # Create an yhat of zeros with the same shape
binary_array[np.arange(yhat.shape[0]), np.argmax(yhat, axis=1)] = 1

print(binary_array)

acc = accuracy_score(test_Y, binary_array)
f1 = f1_score(test_Y,binary_array, average='weighted')
print(f'Accuracy: {acc}, F1: {f1}')

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
rs, maes, rmses = [], [], []


def process_Y(Y):
    Y = np.array(Y)
    Y = np.clip(Y, 0, 100)
    if Y > reaxys_mean + reaxys_std/2:
        return [1,0,0]
    elif Y < reaxys_mean - reaxys_std/2:
        return [0,0,1]
    else:
        return [0,1,0]
    

for i in range(5):
    reaction = False
    train_ids = set(random.sample(list(train_ids0), int(len(train_ids0) * 0.9)))
    val_ids = train_ids0 - train_ids
    test_u = test_ids_u
    # test_u = test_u[int(len(test_u)*0.9):]
    train_X, train_Y = get_raw_feature(train_ids, reaction)
    test_X, test_Y = get_raw_feature(test_ids, reaction)
    train_Y = np.array([process_Y(y) for y in train_Y])
    test_Y = np.array([process_Y(y) for y in test_Y])

    # preparing model
    hyper_parameters = {
        'n_estimators': 409,
        'max_features': 0.124,
        'max_depth': 99
    }
    # wandb.init(project='yield2', entity='oilab', config=hyper_parameters)
    # config = wandb.config
    rg = RandomForestRegressor(n_estimators=hyper_parameters['n_estimators'],
                            max_features= hyper_parameters['max_features'],
                            max_depth=hyper_parameters['max_depth'],
                            n_jobs=16, random_state=0)
    rg.fit(train_X, train_Y)

    # check the performance on the test set
    binary_array = np.zeros_like(yhat)  # Create an yhat of zeros with the same shape
    binary_array[np.arange(yhat.shape[0]), np.argmax(yhat, axis=1)] = 1

    acc = accuracy_score(test_Y, binary_array)
    f1 = f1_score(test_Y,binary_array, average='weighted')
    print(f'Accuracy: {acc}, F1: {f1}')

## test result on unnormal dataset

In [ ]:
rs, maes, rmses = [], [], []
for i in range(5):
    reaction = False
    train_ids = set(random.sample(list(train_ids0), int(len(train_ids0) * 0.9)))
    val_ids = train_ids0 - train_ids
    test_u = test_ids_u
    # test_u = test_u[int(len(test_u)*0.9):]
    train_X, train_Y = get_raw_feature(train_ids, reaction)
    test_X, test_Y = get_raw_feature(test_ids, reaction)
    val_X, val_Y = get_raw_feature(val_ids, reaction)
    test_X_u, test_Y_u = get_raw_feature(test_u, reaction)
    # train_X = rxns.get_fp(list(train_ids))
    # train_Y = np.array([rxns.get_yield(idx) for idx in train_ids])
    train_Y = (train_Y - reaxys_mean) / reaxys_std
    

    # val_X = rxns.get_fp(list(val_ids))
    # val_Y = np.array([rxns.get_yield(idx) for idx in val_ids])

    # test_X = rxns.get_fp(list(test_ids))
    # test_Y = np.array([rxns.get_yield(idx) for idx in test_ids])
    print(f'Training feature shape: {train_X.shape}')
    print(f'Training label shape: {train_Y.shape}')
    print(f'Testing feature shape: {test_X_u.shape}')
    print(f'Testing label shape: {test_Y_u.shape}')

    # preparing model
    hyper_parameters = {
        'n_estimators': 409,
        'max_features': 0.124,
        'max_depth': 99
    }
    # wandb.init(project='yield2', entity='oilab', config=hyper_parameters)
    # config = wandb.config
    rg = RandomForestRegressor(n_estimators=hyper_parameters['n_estimators'],
                            max_features= hyper_parameters['max_features'],
                            max_depth=hyper_parameters['max_depth'],
                            n_jobs=16, random_state=0)
    rg.fit(train_X, train_Y)

    # check the performance on the validation set
    val_y = rg.predict(val_X) * reaxys_std + reaxys_mean
    val_r2 = r2_score(val_Y, val_y)
    val_mae = mean_absolute_error(val_Y, val_y)
    val_rmse = mean_squared_error(val_Y, val_y, squared=False)
    # wandb.log({'val_r2': val_r2, 'val_mae': val_mae, 'val_rmse': val_rmse})
    print(f'Val R2: {val_r2:.3f}')
    print(f'Val MAE: {val_mae:.3f}')
    print(f'Val RMSE: {val_rmse:.3f}')

    # check the performance on the test set
    yhat = rg.predict(test_X_u) * reaxys_std + reaxys_mean
    r2 = r2_score(test_Y_u, yhat)
    mae = mean_absolute_error(test_Y_u, yhat)
    rmse = mean_squared_error(test_Y_u, yhat, squared=False)

    # wandb.log({'r2': r2, 'mae': mae, 'rmse': rmse})
    print(f'R2: {r2:.3f}')
    print(f'MAE: {mae:.3f}')
    print(f'RMSE: {rmse:.3f}')

    # # save data ids, model and predictions
    # pickle.dump(train_ids, open(os.path.join(params_folder, f'RF/{i}_v2_train_ids.pkl'), 'wb'))
    # df_predictions = pd.DataFrame({'id': list(test_ids), 'true': test_Y.tolist(), 'pred': yhat.tolist()})
    # df_predictions.to_csv(os.path.join(params_folder, f'RF/{i}_v2_predictions.csv'), index=False)
    # pickle.dump(rg, open(os.path.join(params_folder, f'RF/{i}_v2_model.pkl'), 'wb'))

    rs.append(r2)
    maes.append(mae)
    rmses.append(rmse)
print(f'R2 mean {np.mean(rs):.3f} std {np.std(rs):.3f}')
print(f'MAE mean {np.mean(maes):.3f} std {np.std(maes):.3f}')
print(f'RMSE mean {np.mean(rmses):.3f} std {np.std(rmses):.3f}')

## test result on clean set

In [ ]:
rs, maes, rmses = [], [], []
for i in range(5):
    reaction = False
    train_ids = set(random.sample(list(train_ids0), int(len(train_ids0) * 0.9)))
    val_ids = train_ids0 - train_ids
    test_u = list(test_ids_u)
    test_u = test_u[int(len(test_u)*1):]
    train_X, train_Y = get_raw_feature(train_ids, reaction)
    test_X, test_Y = get_raw_feature(test_ids, reaction)
    val_X, val_Y = get_raw_feature(val_ids, reaction)
    test_X_u, test_Y_u = get_raw_feature(test_u, reaction)
    # train_X = rxns.get_fp(list(train_ids))
    # train_Y = np.array([rxns.get_yield(idx) for idx in train_ids])
    train_Y = (train_Y - reaxys_mean) / reaxys_std
    

    # val_X = rxns.get_fp(list(val_ids))
    # val_Y = np.array([rxns.get_yield(idx) for idx in val_ids])

    # test_X = rxns.get_fp(list(test_ids))
    # test_Y = np.array([rxns.get_yield(idx) for idx in test_ids])
    print(f'Training feature shape: {train_X.shape}')
    print(f'Training label shape: {train_Y.shape}')
    print(f'Testing feature shape: {test_X.shape}')
    print(f'Testing label shape: {test_Y.shape}')

    # preparing model
    hyper_parameters = {
        'n_estimators': 409,
        'max_features': 0.124,
        'max_depth': 99
    }
    # wandb.init(project='yield2', entity='oilab', config=hyper_parameters)
    # config = wandb.config
    rg = RandomForestRegressor(n_estimators=hyper_parameters['n_estimators'],
                            max_features= hyper_parameters['max_features'],
                            max_depth=hyper_parameters['max_depth'],
                            n_jobs=16, random_state=0)
    rg.fit(train_X, train_Y)

    # check the performance on the validation set
    val_y = rg.predict(val_X) * reaxys_std + reaxys_mean
    val_r2 = r2_score(val_Y, val_y)
    val_mae = mean_absolute_error(val_Y, val_y)
    val_rmse = mean_squared_error(val_Y, val_y, squared=False)
    # wandb.log({'val_r2': val_r2, 'val_mae': val_mae, 'val_rmse': val_rmse})
    print(f'Val R2: {val_r2:.3f}')
    print(f'Val MAE: {val_mae:.3f}')
    print(f'Val RMSE: {val_rmse:.3f}')

    # check the performance on the test set
    yhat = rg.predict(test_X) * reaxys_std + reaxys_mean
    r2 = r2_score(test_Y, yhat)
    mae = mean_absolute_error(test_Y, yhat)
    rmse = mean_squared_error(test_Y, yhat, squared=False)

    # wandb.log({'r2': r2, 'mae': mae, 'rmse': rmse})
    print(f'R2: {r2:.3f}')
    print(f'MAE: {mae:.3f}')
    print(f'RMSE: {rmse:.3f}')

    # # save data ids, model and predictions
    # pickle.dump(train_ids, open(os.path.join(params_folder, f'RF/{i}_v2_train_ids.pkl'), 'wb'))
    # df_predictions = pd.DataFrame({'id': list(test_ids), 'true': test_Y.tolist(), 'pred': yhat.tolist()})
    # df_predictions.to_csv(os.path.join(params_folder, f'RF/{i}_v2_predictions.csv'), index=False)
    # pickle.dump(rg, open(os.path.join(params_folder, f'RF/{i}_v2_model.pkl'), 'wb'))

    rs.append(r2)
    maes.append(mae)
    rmses.append(rmse)
print(f'R2 mean {np.mean(rs):.3f} std {np.std(rs):.3f}')
print(f'MAE mean {np.mean(maes):.3f} std {np.std(maes):.3f}')
print(f'RMSE mean {np.mean(rmses):.3f} std {np.std(rmses):.3f}')